In [5]:
import keras
import tensorflow as tf
from keras import layers
from keras import callbacks
from keras.src.legacy.preprocessing.image import ImageDataGenerator

In [4]:
# Specify GPU usage
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
# Normalize image color values
datagen = ImageDataGenerator(rescale = 1.0 / 255.0)

# Specify path, target shape, batch size, classifier output, color mode, and shuffle
train_generator = datagen.flow_from_directory(
    '../data/train',
    target_size = (32, 32),
    batch_size = 32,
    class_mode = 'binary',
    color_mode = 'rgb',
    shuffle = True
)

val_generator = datagen.flow_from_directory(
    '../data/test',
    target_size = (32, 32),
    batch_size = 32,
    class_mode = 'binary',
    color_mode = 'rgb',
    shuffle = False
)

Found 25000 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In [7]:
########### CODE PROVIDED BY PROFESSOR ###########
model = keras.Sequential([
	keras.Input(shape=(32, 32, 3)),         # Explicit Input Layer
	layers.Conv2D(32, (3,3), activation="relu", padding="same"),
	layers.MaxPooling2D((2,2)),
	layers.Conv2D(64, (3,3), activation="relu", padding="same"),
	layers.MaxPooling2D((2,2)),
	layers.Conv2D(128, (3,3), activation="relu", padding="same"),
	layers.Flatten(),
	layers.Dense(128, activation="relu"),
	layers.Dropout(0.5),                    # Dropout for regularization
	layers.Dense(10, activation="binary")   # Output layer (2 classes)
])

I0000 00:00:1738806552.472052   10756 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1738806552.472093   10756 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
########### CODE PROVIDED BY PROFESSOR ###########
model.compile(
	optimizer="adam",
	# loss="categorical_crossentropy",
    loss="binary_crossentropy",
	metrics=["accuracy"]
)

In [ ]:
# Setup callbacks
early_stopping = callbacks.EarlyStopping(monitor = "val_loss", patience = 5, restore_best_weights = True)
model_checkpoint = callbacks.ModelCheckpoint('best_model.h5', monitor = 'val_loss', save_best_only = True)

In [ ]:
model.fit(
    train_generator,
    epochs = 10,
    validation_data = val_generator,
    callbacks = [early_stopping , model_checkpoint]
)

In [ ]:
########### CODE PROVIDED BY PROVESSOR ###########
model.summary()